<a href="https://colab.research.google.com/github/moosemorse/AI_Text_Detector/blob/main/TATG_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
#gets rid of installation dialogue
%%capture
!pip install transformers
!pip install pytorch
!pip install datasets
!pip install pytorch_lightning

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
import os
import matplotlib.pyplot as plt
from google.colab import files, drive
from datasets import load_dataset
import pandas as pd
import seaborn as sns
import torch
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer
import numpy as np
import copy


In [ ]:
#mount drive, gain access to file in google drive
drive.mount('/content/drive', force_remount=False)

#obtain csv file and store in var 'df' as dataframe
path = "drive/MyDrive/GPT-wiki-intro.csv"
df = pd.read_csv(path)

# Inspect data

In [ ]:
#data to describe csv file
print(df.describe())

In [ ]:
df.head()

In [ ]:
print(df.iloc[:].loc[:, ['wiki_intro', 'generated_intro']])
#iloc dictates the rows indexed
#loc dictates the columns extracted
#dataset balanced in examples of gpt vs human
length_df = int(len(df))
print(df.iloc[0:int(length_df/2)]['generated_intro'])

In [ ]:
#visualisation to compare data for human-written text and ai-written text
#testing seaborn and these could be helpful for evaluation afterwards
sns.countplot(x = 'wiki_intro_len', data = df)
plt.show()

sns.countplot(x = 'generated_intro_len', data = df)
plt.show()

#this also implies we must clean the data in order to make
#the dataset more balanced to remove bias from one class if text is too short
#also this improves performance of the classifier for longer texts - which is what it will predominantly be used for

In [ ]:
df.max()

In [ ]:
#actually 300,000 examples since a generated message and wiki message
#are on same row, each similar in topic
len(df)

# Dataset

to create a useful chatGPT dataset (which will be returning tensors):

In [ ]:
#dataset class inherits dataset module imported from torch
class ChatGPT_Dataset(Dataset):

  def __init__(self, data_path, stage, tokenizer, max_token_len = 512):
    self.data_path = data_path
    self.tokenizer = tokenizer
    self.stage = stage
    self.max_token_len = max_token_len
    self._prepare_data(stage)

  #cleans dataframe to create dataset with text needed and labels
  #1 represents human written, 0 represents generated
  def _prepare_data(self, stage):
    #80/20 split for training/test
    data = pd.read_csv(self.data_path)
    if stage == 'train':
      #dataframe for generated data, additional column label added
      generated = pd.DataFrame({'text': data.iloc[0:int(len(data)*0.8)]['generated_intro'], 'label': 0})
      #dataframe for human-written data, additional column label added
      wiki = pd.DataFrame({'text': data.iloc[0:int(len(data)*0.8)]['wiki_intro'], 'label': 1})
      #concatenate both dataframes
      self.data = pd.concat([generated, wiki])
    if stage == 'test':
      generated = pd.DataFrame({'text': data.iloc[int(len(data)*0.8):]['generated_intro'], 'label': 0})
      wiki = pd.DataFrame({'text': data.iloc[int(len(data)*0.8):]['wiki_intro'], 'label': 1})
      self.data = pd.concat([generated, wiki])
    if stage == 'predict':
      self.data = data

  def __len__(self):
    return (len(self.data))

  def __getitem__(self, index):
    #find row in data which contains text and label
    item = self.data.iloc[index]
    message = str(item.text)
    label = torch.LongTensor([item['label']])
    #tokenize our message, returning tensors for input ids and an attention mask
    #truncation and padding used so all tensors are the same size
    tokens = self.tokenizer.encode_plus(message,
                                        add_special_tokens = True,
                                        return_tensors ='pt',
                                        truncation = True,
                                        max_length = self.max_token_len,
                                        padding = 'max_length',
                                        return_attention_mask = True)

    return {'input_ids': tokens.input_ids.flatten(), 'attention_mask': tokens.attention_mask.flatten(),
            'labels': label }

In [ ]:
#use pretrained tokenizer used from Roberta
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
training_ChatGPT_ds = ChatGPT_Dataset(path, tokenizer = tokenizer, stage = 'train')
testing_ChatGPT_ds = ChatGPT_Dataset(path, tokenizer = tokenizer, stage = 'test')
print(training_ChatGPT_ds.__getitem__(0))
print(testing_ChatGPT_ds.__getitem__(0))

In [ ]:
#correct number of total messages since 300,000 * 0.8 = 240,000
len(training_ChatGPT_ds)

In [ ]:
#correct number of total messages since 300,000 * 0.2 = 60,000
len(testing_ChatGPT_ds)

Create a data module to create datasets for training data set and its also going to return the data loaders.

In [ ]:
class ChatGPT_Data_Module(pl.LightningDataModule):

  def __init__(self, path, batch_size = 16, max_token_len = 512, model_name = 'roberta-base'):
    super().__init__()
    self.path = path
    self.batch_size = batch_size
    self.max_token_len = max_token_len
    self.model_name = model_name
    self.tokenizer = RobertaTokenizer.from_pretrained(model_name)

  #create dataset (defined in previous class)
  def setup(self, stage = None):
    if stage in (None, "train"):
      self.train_dataset = ChatGPT_Dataset(self.path, tokenizer = self.tokenizer, stage ="train")
    if stage == 'test':
      self.test_dataset = ChatGPT_Dataset(self.path, tokenizer = self.tokenizer, stage = "test")

  #return dataloader for training dataset
  #'num_workers' denote number of processes that generate batches in parallel
  #'shuffle = True' to randomly choose items
  def train_dataloader(self):
    return DataLoader(self.train_dataset, batch_size = self.batch_size, num_workers = 2, shuffle = True)

  def test_dataloader(self):
    return DataLoader(self.test_dataset, batch_size = self.batch_size, num_workers = 2, shuffle = False)

In [ ]:
training_ChatGPT_data_module = ChatGPT_Data_Module(path)

In [ ]:
training_ChatGPT_data_module.setup(stage = "train")

In [ ]:
dl = training_ChatGPT_data_module.train_dataloader()

In [ ]:
len(dl)
#output correct: 240,000 / 16 = 15,000

15000

# Creating the model

In [ ]:
from transformers import RobertaConfig, RobertaModel, AdamW, get_cosine_schedule_with_warmup
import torch.nn as nn
import math
from torchmetrics.functional.classification import auroc
import torch.nn.functional as F

In [ ]:
class ChatGPT_Classifier(pl.LightningModule):
  #roberta is a pretrained model that can be used for many downstream task
  #in our context we are using it for classification so we'd want to append a classification head onto the end of the model
  #to improve our performance even more, we will add a 2 layer neural network to the end (so a hidden layer --> final layer)
  def __init__(self, config):
    super().__init__()
    self.config = config
    self.pretrained_model = RobertaModel.from_pretrained('roberta-base', return_dict = True)
    self.hidden = nn.Linear(self.pretrained_model.config.hidden_size, self.pretrained_model.config.hidden_size) #hidden layer
    self.classifier = nn.Linear(self.pretrained_model.config.hidden_size, self.config['n_labels']) #classification layer
    #torch can automatically initialise these layers, but using xavier_uniform
    #means we can initialise the weight of the NN layer ==> improving performance
    torch.nn.init.xavier_uniform_(self.hidden.weight)
    torch.nn.init.xavier_uniform_(self.classifier.weight)
    #loss function - Binary cross entropy with logits loss to pass in our output
    #labels to create a single loss which we can then backpropogate to our network
    #BCEwithlogitsloss is more numerically stable - it combines a sigmoid layer and BCE in a single class
    self.loss_func = nn.BCEWithLogitsLoss(reduction='mean')
    #dropout layer - randomly turns on/off several nodes in NN every iteration,
    #as a form of some regularization
    self.dropout = nn.Dropout()

  def forward(self, input_ids, attention_mask, labels=None):
    #roberta model
    output = self.pretrained_model(input_ids = input_ids, attention_mask = attention_mask)
    #use a mean output as its a better representation of the entire sentence
    #dimension we take the mean on is the first dimension
    #since this is the tokens we have
    pooled_output = torch.mean(output.last_hidden_state, 1)
    #nerual network classification layers
    #pass pooled_output through hidden layer
    pooled_output = self.hidden(pooled_output)
    #pass pooled_output into dropout layer which forces the model to try
    #classify a sentence with only a few tokens left
    pooled_output = self.dropout(pooled_output)
    #pass pooled_output through activation function (relu)
    pooled_output = F.relu(pooled_output)
    #final output (=logits)
    logits = self.classifier(pooled_output)
    #calculate loss
    loss = 0
    #if labels are present this means that we are using training data
    #hence a loss is calculated
    if labels is not None:
      labels = labels.to(logits.dtype)
      loss = self.loss_func(logits.view(-1, self.config['n_labels']), labels.view(-1, self.config['n_labels']) )
    return loss, logits

  def training_step(self, batch, batch_index):
    loss, logits = self(**batch)
    #-----comments needed-------
    self.log("train loss", loss, prog_bar = True, logger = True)
    return {"loss": loss, "predictions": logits, "labels": batch['labels']}

  def testing_step(self, batch, batch_index):
    loss, logits = self(**batch)
    #-----comments needed-------
    self.log("test loss", loss, prog_bar = True, logger = True)
    return {"test_loss": loss, "predictions": logits, "labels": batch['labels']}

  def predict_step(self, batch, batch_index):
    #unpack contents of dictionary (batch) and pass in as kwargs
    none, logits = self(**batch)
    return logits

  def configure_optimizers(self):
    optimiser = AdamW(self.parameters(), lr=self.config['lr'], weight_decay = self.config['w_decay'])
    #train_size/batch size
    total_steps = self.config['train_size'] / self.config['bs']
    #this is needed as we a have a warmup period for the
    warmup_steps = math.floor(total_steps *  self.config['warmup'])
    scheduler = get_cosine_schedule_with_warmup(optimiser, warmup_steps, total_steps)
    return [optimiser], [scheduler]

In [ ]:
#hyperparameters
config = {
    'n_labels': 1,
    'bs': 128,
    'lr' : 1.5e-6,
    'warmup' : 0.2,
    'train_size': len(training_ChatGPT_data_module.train_dataloader()),
    'w_decay': 0.001,
    'n_epochs': 5
}

model = ChatGPT_Classifier(config)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#testing if model works
idx = 0
input_ids = training_ChatGPT_ds.__getitem__(idx)['input_ids']
am = training_ChatGPT_ds.__getitem__(idx)['attention_mask']
label = training_ChatGPT_ds.__getitem__(idx)['labels']
#unsqueeze needed because the model is expecting a batch
#so an extra dimension is needed since only one sample is returned
loss, output = model(input_ids.unsqueeze(dim = 0), am.unsqueeze(dim=0), label.unsqueeze(dim=0) )

In [ ]:
print(loss,output)

tensor(1.3371, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) tensor([[1.0325]], grad_fn=<AddmmBackward0>)


#Training the model

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 1            |        cudaMalloc retries: 1         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |  14321 MiB |  14748 MiB |  27821 MiB |  13500 MiB |\n|       from large pool |  14317 MiB |  14742 MiB |  27798 MiB |  13481 MiB |\n|       from small pool |      4 MiB |      5 MiB |     23 MiB |     19 MiB |\n|---------------------------------------------------------------------------|\n| Active memory         |  14321 MiB |  14748 MiB |  27821 MiB |  13500 MiB |\n|       from large pool |  14317 MiB |  14742 MiB |

In [ ]:
#datamodule
training_chatgpt_data_module = ChatGPT_Data_Module(path, batch_size = config['bs'])
training_chatgpt_data_module.setup(stage = "train")

#model
model = ChatGPT_Classifier(config)

#train
trainer = pl.Trainer(max_epochs=config['n_epochs'], accelerator = 'gpu')
trainer.fit(model, training_chatgpt_data_module)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings

Training: 0it [00:00, ?it/s]

OutOfMemoryError: ignored

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./lightning_logs/

# Evaluating the model